In [2]:
!pip install gradio

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from scipy.optimize import minimize
import gradio as gr

# -SIMULAR Y ENTRENAR MODELOS
# (Este proceso se hace una sola vez al inicio para tener los modelos listos)
np.random.seed(42)
num_anios = 50
anios = np.arange(2015, 2015 + num_anios)
lluvia_simulada = np.random.normal(loc=700, scale=100, size=num_anios)
temp_media_simulada = np.random.normal(loc=20, scale=3, size=num_anios)
estado_enso = np.random.choice([-1, 0, 1], size=num_anios)
gdd_acumulado = np.random.normal(loc=2000, scale=200, size=num_anios)
ndvi_acumulado = np.random.normal(loc=5, scale=0.8, size=num_anios)

# Generar rendimientos con relaciones realistas
maiz_simulado = 6500 + (lluvia_simulada - 700) * 5 + (temp_media_simulada - 20) * 50 + np.random.randint(-400, 400, num_anios)
choclo_simulado = 8000 + (lluvia_simulada - 700) * 4 + (temp_media_simulada - 20) * 80 + np.random.randint(-450, 450, num_anios)
zapallo_simulado = 11000 + (lluvia_simulada - 700) * 2 + (temp_media_simulada - 20) * 100 + np.random.randint(-150, 150, num_anios)

df_final = pd.DataFrame({
    'YEAR': anios, 'lluvia_acumulada_anual': lluvia_simulada,
    'temp_media_anual': temp_media_simulada, 'estado_enso': estado_enso,
    'gdd_acumulado': gdd_acumulado, 'ndvi_acumulado': ndvi_acumulado,
    'Maíz': maiz_simulado, 'Choclo': choclo_simulado, 'Zapallo': zapallo_simulado
})

X = df_final[['lluvia_acumulada_anual', 'temp_media_anual', 'estado_enso', 'gdd_acumulado', 'ndvi_acumulado']]
y_maiz = df_final['Maíz']
y_choclo = df_final['Choclo']
y_zapallo = df_final['Zapallo']
modelo_maiz = RandomForestRegressor(n_estimators=100, random_state=42).fit(X, y_maiz)
modelo_choclo = RandomForestRegressor(n_estimators=100, random_state=42).fit(X, y_choclo)
modelo_zapallo = RandomForestRegressor(n_estimators=100, random_state=42).fit(X, y_zapallo)

# FUNCIÓN PRINCIPAL PARA GRADIO
# Esta función toma los parámetros del usuario y ejecuta el modelo completo
def calcular_recomendacion(lluvia, temperatura, enso, gdd, ndvi):
    datos_futuros = pd.DataFrame([{
        'lluvia_acumulada_anual': lluvia,
        'temp_media_anual': temperatura,
        'estado_enso': enso,
        'gdd_acumulado': gdd,
        'ndvi_acumulado': ndvi
    }])

    # Predicción de rendimientos
    rendimiento_predicho_maiz = modelo_maiz.predict(datos_futuros)[0]
    rendimiento_predicho_choclo = modelo_choclo.predict(datos_futuros)[0]
    rendimiento_predicho_zapallo = modelo_zapallo.predict(datos_futuros)[0]
    rendimientos_predichos = np.array([rendimiento_predicho_maiz, rendimiento_predicho_choclo, rendimiento_predicho_zapallo])

    # Variables de la optimización
    precios = np.array([0.2, 0.3, 0.25])
    costos = np.array([0.1, 0.1, 0.1])
    superficie_total_disponible = 1000
    demanda_minima_maiz = 1500
    demanda_minima_choclo = 150
    demanda_minima_zapallo = 100
    umbral_rentabilidad = 150000

    nombres_cultivos = ['Maíz', 'Choclo', 'Zapallo']
    ganancia_por_hectarea = (precios - costos) * rendimientos_predichos
    cultivos_viables_indices = [i for i, ganancia in enumerate(ganancia_por_hectarea) if ganancia > umbral_rentabilidad/superficie_total_disponible]

    nombres_cultivos_viables = [nombres_cultivos[i] for i in cultivos_viables_indices]
    precios_viables = precios[cultivos_viables_indices]
    costos_viables = costos[cultivos_viables_indices]
    rendimientos_viables = rendimientos_predichos[cultivos_viables_indices]
    demanda_minima_viables = [demanda_minima_maiz, demanda_minima_choclo, demanda_minima_zapallo]
    demanda_minima_viables = [demanda_minima_viables[i] for i in cultivos_viables_indices]

    def funcion_ganancia(superficies):
        ganancia_total = np.sum((precios_viables - costos_viables) * rendimientos_viables * superficies)
        return -ganancia_total

    restriccion_superficie = {'type': 'eq', 'fun': lambda superficies: np.sum(superficies) - superficie_total_disponible}
    restricciones_demanda = [{'type': 'ineq', 'fun': lambda s, i=i: (s[i] * rendimientos_viables[i]) - demanda_minima_viables[i]} for i in range(len(cultivos_viables_indices))]
    restricciones = [restriccion_superficie] + restricciones_demanda
    limites_superficie = [(0, None) for _ in range(len(cultivos_viables_indices))]
    adivinanza_inicial = [superficie_total_disponible/len(cultivos_viables_indices)] * len(cultivos_viables_indices)
    resultado = minimize(fun=funcion_ganancia, x0=adivinanza_inicial, bounds=limites_superficie, constraints=restricciones)

    superficies_optimas = resultado.x
    ganancia_maxima = -resultado.fun

    # Formatear el resultado para la interfaz
    resultados = "### 🌾 Predicciones de Rendimiento Futuro\n"
    resultados += f"Maíz: {rendimiento_predicho_maiz:.2f} kg/ha\n"
    resultados += f"Choclo: {rendimiento_predicho_choclo:.2f} kg/ha\n"
    resultados += f"Zapallo: {rendimiento_predicho_zapallo:.2f} kg/ha\n\n"

    resultados += "### 🌱 Recomendación de Siembra Óptima\n"
    for i, cultivo in enumerate(nombres_cultivos_viables):
        resultados += f"**{cultivo}:** {superficies_optimas[i]:.2f} ha\n"

    resultados += f"\n### 💰 Proyección de Ganancia Máxima\n"
    resultados += f"**Ganancia Total Proyectada:** ${ganancia_maxima:,.2f}"

    return resultados

# --- 3. CONFIGURAR LA INTERFAZ DE GRADIO ---
inputs = [
    gr.Slider(minimum=200, maximum=1200, value=600, label="Lluvia Anual Acumulada (mm)"),
    gr.Slider(minimum=15, maximum=25, value=23, label="Temperatura Media Anual (C°)"),
    gr.Slider(minimum=-1, maximum=1, step=1, value=-1, label="Estado ENSO (-1:Niña, 0:Normal, 1:Niño)"),
    gr.Slider(minimum=1500, maximum=2500, value=2100, label="GDD Acumulado"),
    gr.Slider(minimum=3, maximum=7, value=4.5, label="NDVI Acumulado")
]

output = gr.Markdown(label="Recomendación de Siembra y Ganancia Proyectada")

demo = gr.Interface(
    fn=calcular_recomendacion,
    inputs=inputs,
    outputs=output,
    title="Planificador de Siembra Inteligente",
    description="Ajusta los parámetros climáticos para obtener una recomendación de siembra optimizada para el próximo ciclo."
)

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d002f1ae85b6867ce6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [3]:
!pip install gradio

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from scipy.optimize import minimize
import gradio as gr

# --- 1. SIMULAR Y ENTRENAR MODELOS ---
# (Este proceso se hace una sola vez al inicio para tener los modelos listos)
np.random.seed(42)
num_anios = 50
anios = np.arange(2015, 2015 + num_anios)
lluvia_simulada = np.random.normal(loc=700, scale=100, size=num_anios)
temp_media_simulada = np.random.normal(loc=20, scale=3, size=num_anios)
estado_enso = np.random.choice([-1, 0, 1], size=num_anios)
gdd_acumulado = np.random.normal(loc=2000, scale=200, size=num_anios)
ndvi_acumulado = np.random.normal(loc=5, scale=0.8, size=num_anios)

# Generar rendimientos con relaciones realistas
maiz_simulado = 6500 + (lluvia_simulada - 700) * 5 + (temp_media_simulada - 20) * 50 + np.random.randint(-400, 400, num_anios)
choclo_simulado = 8000 + (lluvia_simulada - 700) * 4 + (temp_media_simulada - 20) * 80 + np.random.randint(-450, 450, num_anios)
zapallo_simulado = 11000 + (lluvia_simulada - 700) * 2 + (temp_media_simulada - 20) * 100 + np.random.randint(-150, 150, num_anios)

df_final = pd.DataFrame({
    'YEAR': anios, 'lluvia_acumulada_anual': lluvia_simulada,
    'temp_media_anual': temp_media_simulada, 'estado_enso': estado_enso,
    'gdd_acumulado': gdd_acumulado, 'ndvi_acumulado': ndvi_acumulado,
    'Maíz': maiz_simulado, 'Choclo': choclo_simulado, 'Zapallo': zapallo_simulado
})

X = df_final[['lluvia_acumulada_anual', 'temp_media_anual', 'estado_enso', 'gdd_acumulado', 'ndvi_acumulado']]
y_maiz = df_final['Maíz']
y_choclo = df_final['Choclo']
y_zapallo = df_final['Zapallo']
modelo_maiz = RandomForestRegressor(n_estimators=100, random_state=42).fit(X, y_maiz)
modelo_choclo = RandomForestRegressor(n_estimators=100, random_state=42).fit(X, y_choclo)
modelo_zapallo = RandomForestRegressor(n_estimators=100, random_state=42).fit(X, y_zapallo)


# --- 2. FUNCIÓN PARA ESTIMAR HECTÁREAS DE AUTOCONSUMO ---
def estimar_hectareas(n_personas, n_animales, rendimientos):
    consumo_humano_default = {
        'maiz': 25,
        'zapallo': 15,
        'choclo': 12
    }
    consumo_animal_default = {
        'cabras': 50,
        'ovejas': 60
    }
    reparto_animal = {
        'maiz': 0.8,
    }

    # Calcular la demanda total de la familia
    demanda_total_kg = {
        'maiz': n_personas * consumo_humano_default['maiz'],
        'zapallo': n_personas * consumo_humano_default['zapallo'],
        'choclo': n_personas * consumo_humano_default['choclo']
    }

    # Sumar la demanda del ganado (solo maíz)
    if 'cabras' in n_animales:
        demanda_total_kg['maiz'] += n_animales['cabras'] * consumo_animal_default['cabras'] * reparto_animal.get('maiz', 0)
    if 'ovejas' in n_animales:
        demanda_total_kg['maiz'] += n_animales['ovejas'] * consumo_animal_default['ovejas'] * reparto_animal.get('maiz', 0)

    # Calcular las hectáreas necesarias para cada cultivo
    hectareas_requeridas = {}
    for cultivo, demanda in demanda_total_kg.items():
        if rendimientos[cultivo] > 0:
            hectareas_requeridas[cultivo] = demanda / rendimientos[cultivo]
        else:
            hectareas_requeridas[cultivo] = 0

    return hectareas_requeridas

# --- 3. FUNCIÓN PRINCIPAL PARA GRADIO ---
def calcular_recomendacion(lluvia, temperatura, enso, gdd, ndvi, n_personas, n_cabras, n_ovejas):
    datos_futuros = pd.DataFrame([{
        'lluvia_acumulada_anual': lluvia,
        'temp_media_anual': temperatura,
        'estado_enso': enso,
        'gdd_acumulado': gdd,
        'ndvi_acumulado': ndvi
    }])

    # Predicción de rendimientos
    rendimiento_predicho_maiz = modelo_maiz.predict(datos_futuros)[0]
    rendimiento_predicho_choclo = modelo_choclo.predict(datos_futuros)[0]
    rendimiento_predicho_zapallo = modelo_zapallo.predict(datos_futuros)[0]
    rendimientos_predichos = {
        'maiz': rendimiento_predicho_maiz,
        'choclo': rendimiento_predicho_choclo,
        'zapallo': rendimiento_predicho_zapallo
    }

    # Calcular las hectáreas de autoconsumo con los rendimientos predichos
    n_animales = {'cabras': n_cabras, 'ovejas': n_ovejas}
    hectareas_autoconsumo = estimar_hectareas(n_personas, n_animales, rendimientos_predichos)

    # Variables de la optimización (ahora incluyen la demanda mínima de autoconsumo)
    precios = np.array([0.2, 0.3, 0.25])
    costos = np.array([0.1, 0.1, 0.1])
    superficie_total_disponible = 1000

    # La demanda mínima es lo que se requiere para autoconsumo
    demanda_minima_maiz = hectareas_autoconsumo['maiz'] * rendimientos_predichos['maiz']
    demanda_minima_choclo = hectareas_autoconsumo['choclo'] * rendimientos_predichos['choclo']
    demanda_minima_zapallo = hectareas_autoconsumo['zapallo'] * rendimientos_predichos['zapallo']

    umbral_rentabilidad = 150000

    nombres_cultivos = ['Maíz', 'Choclo', 'Zapallo']
    ganancia_por_hectarea = (precios - costos) * np.array(list(rendimientos_predichos.values()))
    cultivos_viables_indices = [i for i, ganancia in enumerate(ganancia_por_hectarea) if ganancia > umbral_rentabilidad / superficie_total_disponible]

    nombres_cultivos_viables = [nombres_cultivos[i] for i in cultivos_viables_indices]
    precios_viables = precios[cultivos_viables_indices]
    costos_viables = costos[cultivos_viables_indices]
    rendimientos_viables = np.array(list(rendimientos_predichos.values()))[cultivos_viables_indices]
    demanda_minima_viables = [demanda_minima_maiz, demanda_minima_choclo, demanda_minima_zapallo]
    demanda_minima_viables = [demanda_minima_viables[i] for i in cultivos_viables_indices]

    # Función objetivo y restricciones
    def funcion_ganancia(superficies):
        ganancia_total = np.sum((precios_viables - costos_viables) * rendimientos_viables * superficies)
        return -ganancia_total

    restriccion_superficie = {'type': 'eq', 'fun': lambda superficies: np.sum(superficies) - superficie_total_disponible}
    restricciones_demanda = [{'type': 'ineq', 'fun': lambda s, i=i: (s[i] * rendimientos_viables[i]) - demanda_minima_viables[i]} for i in range(len(cultivos_viables_indices))]
    restricciones = [restriccion_superficie] + restricciones_demanda
    limites_superficie = [(0, None) for _ in range(len(cultivos_viables_indices))]
    adivinanza_inicial = [superficie_total_disponible/len(cultivos_viables_indices)] * len(cultivos_viables_indices)
    resultado = minimize(fun=funcion_ganancia, x0=adivinanza_inicial, bounds=limites_superficie, constraints=restricciones)

    superficies_optimas = resultado.x
    ganancia_maxima = -resultado.fun

    # Formatear el resultado para la interfaz
    resultados = "### 🌾 Predicciones de Rendimiento Futuro\n"
    resultados += f"Maíz: {rendimientos_predichos['maiz']:.2f} kg/ha\n"
    resultados += f"Choclo: {rendimientos_predichos['choclo']:.2f} kg/ha\n"
    resultados += f"Zapallo: {rendimientos_predichos['zapallo']:.2f} kg/ha\n\n"

    resultados += f"### 🏠 Hectáreas de Autoconsumo (para {n_personas} personas y {n_cabras} cabras, {n_ovejas} ovejas)\n"
    for cultivo, ha in hectareas_autoconsumo.items():
        resultados += f"**{cultivo.capitalize()}:** {ha:.2f} ha\n"

    resultados += "\n### 🌱 Recomendación de Siembra Total\n"
    for i, cultivo in enumerate(nombres_cultivos_viables):
        resultados += f"**{cultivo}:** {superficies_optimas[i]:.2f} ha\n"

    resultados += f"\n### 💰 Proyección de Ganancia Máxima\n"
    resultados += f"**Ganancia Total Proyectada:** ${ganancia_maxima:,.2f}"

    return resultados

# --- 4. CONFIGURAR LA INTERFAZ DE GRADIO ---
inputs = [
    gr.Slider(minimum=200, maximum=1200, value=600, label="Lluvia Anual Acumulada (mm)"),
    gr.Slider(minimum=15, maximum=25, value=23, label="Temperatura Media Anual (C°)"),
    gr.Slider(minimum=-1, maximum=1, step=1, value=-1, label="Estado ENSO (-1:Niña, 0:Normal, 1:Niño)"),
    gr.Slider(minimum=1500, maximum=2500, value=2100, label="GDD Acumulado"),
    gr.Slider(minimum=3, maximum=7, value=4.5, label="NDVI Acumulado"),
    gr.Slider(minimum=1, maximum=20, step=1, value=4, label="Número de Personas"),
    gr.Slider(minimum=0, maximum=50, step=1, value=10, label="Número de Cabras"),
    gr.Slider(minimum=0, maximum=50, step=1, value=5, label="Número de Ovejas")
]

output = gr.Markdown(label="Recomendación de Siembra y Ganancia Proyectada")

demo = gr.Interface(
    fn=calcular_recomendacion,
    inputs=inputs,
    outputs=output,
    title="Planificador de Siembra Inteligente",
    description="Ajusta los parámetros climáticos y de consumo para obtener una recomendación de siembra optimizada para el próximo ciclo."
)

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://259c0217b48bf3f679.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


1. Entrada de Datos (La Interfaz de Gradio)
Cuando el productor mueve los controles deslizantes y hace clic en Submit, lo que hace es decirle al modelo las condiciones climáticas. Los valores que elige para la lluvia, temperatura, GDD, etc., son las entradas que el modelo necesita para hacer su pronóstico. Es la forma en que el productor interactúa con la herramienta.

2. El Cerebro de la IA (El Modelo de Predicción y Optimización)
Al hacer clic en Submit( Entrega), el código entra en acción. El programa toma los valores de los controles deslizantes y los usa en dos etapas:

Predicción de Rendimiento: Primero, alimenta estos datos a los modelos de Random Forest que entrenaste previamente. Estos modelos analizan las condiciones y te dan una predicción de cuántos kilogramos por hectárea se espera que produzca cada cultivo (Maíz, Choclo y Zapallo).

Optimización: Luego, la "calculadora" entra en juego. El programa toma esas predicciones de rendimiento y, usando una fórmula de optimización, calcula la mejor combinación de hectáreas para cada cultivo. Aquí es donde se aplican todas las reglas que definimos: cubrir las necesidades de autoconsumo, destinar el resto del terreno al cultivo más rentable y maximizar la ganancia total.

3. La Recomendación  (El Resultado)
el resultado de la optimización se formatea en un texto simple y claro, que es lo que ve en la pantalla. Muestra el rendimiento predicho, la cantidad de hectáreas recomendadas para sembrar y la ganancia proyectada.

En esencia, la interfaz de Gradio es como el tablero de control de un auto. Tú ajustas los "sensores" (los controles deslizantes), y el "motor de IA" (el código) hace todas las complejas operaciones internas para darte una respuesta clara y útil.

ahora es un Producto Mínimo Viable (MVP) que demuestra la idea central del proyecto. La entrada manual de datos mediante los controles deslizantes es una forma simple de mostrar el concepto y el valor de la herramienta.